## Importing Data and Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler, PowerTransformer, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import scipy.stats as stats
from sklearn.feature_selection import mutual_info_classif
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/My Drive/PGPDSE/Capstone/CSV Files/diabetes_50k_for_feature_selection.csv')
df.head()

,race,gender,age,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,diag_1,diag_2,diag_3,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,glyburide-metformin,diabetesMed,readmitted,preceding_year_visits,number_changes,insulin_treatment
0,Caucasian,1,85,Transferred to another medical facility,Transferred from another health care facility,4,56,0,4,Genitourinary,Infectious and Parasitic,"Endocrine, Nutritional, Metabolic, Immunity",9,None,None,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,1,0,1,0,other_meds
1,Caucasian,1,75,Discharged to home,Referral,1,46,3,13,Circulatory,Circulatory,Circulatory,8,None,None,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0,no_med
2,AfricanAmerican,1,55,Not Available,Emergency,5,52,1,23,Musculoskeletal System and Connective Tissue,Infectious and Parasitic,Mental Disorders,5,None,None,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,1,0,2,0,insulin_only
3,Caucasian,1,65,Discharged to home,Referral,5,27,2,28,Neoplasms,Neoplasms,Respiratory,8,None,None,-2,-2,-2,-2,-2,1,-2,-2,0,-2,1,0,0,1,insulin_combo
4,Caucasian,0,85,Transferred to another medical facility,Referral,11,73,0,23,Circulatory,Circulatory,Circulatory,9,None,>8,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,1,0,3,1,insulin_only


In [ ]:
df.shape

(50000, 30)

## Preparing Data

In [ ]:
X = df.drop('readmitted', 1)
X_dum = pd.get_dummies(X, drop_first = True)
y = df['readmitted']
X.shape, X_dum.shape, y.shape

((50000, 29), (50000, 97), (50000,))

In [ ]:
X_dum.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X_dum.columns]

In [ ]:
models = []

models.append(('LGB',LGBMClassifier(random_state = 0, n_jobs = -1)))
models.append(('LR', Pipeline([("Transformer", PowerTransformer()),
                               ("Scaler", StandardScaler()),
                               ("LogReg", LogisticRegression(random_state = 0, solver = 'liblinear', n_jobs = -1))])))
models.append(('KNN', Pipeline([("Transformer", PowerTransformer()),
                               ("Scaler", StandardScaler()),
                               ("KNN", KNeighborsClassifier(n_jobs = -1))])))
models.append(('DT', DecisionTreeClassifier(random_state = 0)))
models.append(('BC',BaggingClassifier(random_state = 0, n_jobs = -1)))
models.append(('ET',ExtraTreesClassifier(random_state = 0, n_jobs = -1)))
models.append(('RF', RandomForestClassifier(random_state = 0, n_jobs = -1)))
models.append(('ADA',AdaBoostClassifier(random_state = 0)))
models.append(('GB',GradientBoostingClassifier(random_state = 0)))
models.append(('XGB', XGBClassifier(random_state = 0, n_jobs = -1)))

## Sequential Forward Selection

In [ ]:
X_dum.shape

(50000, 97)

In [ ]:
model = LGBMClassifier(random_state = 0, n_jobs = -1)

# Create an SFS object
sfs = SFS(estimator = model,
          k_features = 17,
          forward = True,       # Set forward to True when we want to perform SFS
          scoring = 'recall',
          cv = 5)               # The number of cross-validations to perform is 5

# Train SFS with our dataset
sfs = sfs.fit(X_dum, y)

In [ ]:
# Show the performance of each subset of features considered by SFS
sfs_results = pd.DataFrame.from_dict(sfs.subsets_).T.sort_values(by = 'avg_score', ascending = False) 
sfs_results

,feature_idx,cv_scores,avg_score,feature_names
17,"(6, 17, 18, 19, 30, 31, 37, 47, 57, 59, 61, 70...","[0.013089005235602094, 0.012216404886561954, 0...",0.0125612,"(number_diagnoses, diabetesMed, preceding_year..."
15,"(6, 17, 18, 19, 30, 37, 47, 57, 61, 70, 73, 82...","[0.013961605584642234, 0.010471204188481676, 0...",0.0123865,"(number_diagnoses, diabetesMed, preceding_year..."
16,"(6, 17, 18, 19, 30, 31, 37, 47, 57, 61, 70, 73...","[0.013089005235602094, 0.008726003490401396, 0...",0.0123862,"(number_diagnoses, diabetesMed, preceding_year..."
14,"(6, 17, 18, 19, 30, 37, 47, 57, 61, 70, 73, 92...","[0.013089005235602094, 0.010471204188481676, 0...",0.0122123,"(number_diagnoses, diabetesMed, preceding_year..."
10,"(6, 17, 18, 19, 30, 47, 57, 61, 70, 95)","[0.013089005235602094, 0.009598603839441536, 0...",0.0118634,"(number_diagnoses, diabetesMed, preceding_year..."
9,"(6, 17, 18, 19, 30, 47, 57, 61, 70)","[0.013089005235602094, 0.009598603839441536, 0...",0.0118634,"(number_diagnoses, diabetesMed, preceding_year..."
13,"(6, 17, 18, 19, 30, 37, 47, 57, 61, 70, 73, 94...","[0.013089005235602094, 0.009598603839441536, 0...",0.0118631,"(number_diagnoses, diabetesMed, preceding_year..."
12,"(6, 17, 18, 19, 30, 37, 47, 57, 61, 70, 73, 95)","[0.013961605584642234, 0.010471204188481676, 0...",0.011689,"(number_diagnoses, diabetesMed, preceding_year..."
11,"(6, 17, 18, 19, 30, 47, 57, 61, 70, 73, 95)","[0.012216404886561954, 0.008726003490401396, 0...",0.0115145,"(number_diagnoses, diabetesMed, preceding_year..."
8,"(6, 17, 18, 19, 30, 47, 57, 61)","[0.012216404886561954, 0.008726003490401396, 0...",0.0113398,"(number_diagnoses, diabetesMed, preceding_year..."


In [ ]:
sfs_results.iloc[0, 3]

('number_diagnoses',
 'diabetesMed',
 'preceding_year_visits',
 'number_changes',
 'admission_source_id_Referral',
 'admission_source_id_Transferred_from_another_health_care_facility',
 'diag_1_External_causes_of_injury',
 'diag_1_Respiratory',
 'diag_2_Infectious_and_Parasitic',
 'diag_2_Mental_Disorders',
 'diag_2_Neoplasms',
 'diag_3_Congenital_Anomalies',
 'diag_3_Endocrine__Nutritional__Metabolic__Immunity',
 'diag_3_Not_Required',
 'A1Cresult_None',
 'insulin_treatment_insulin_only',
 'insulin_treatment_no_med')

In [ ]:
sfs_results.to_csv('/content/drive/My Drive/PGPDSE/Capstone/CSV Files/50K_LGBM_Forward_Results_Saurabh.csv', index = False)

## Sequential Backward Selector

In [ ]:
X_dum.head()

,gender,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_diagnoses,metformin,repaglinide,nateglinide,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,glyburide_metformin,diabetesMed,preceding_year_visits,number_changes,race_Asian,race_Caucasian,race_Hispanic,race_Other,discharge_disposition_id_Discharged_to_home_with_home_health_service,discharge_disposition_id_Left_AMA,discharge_disposition_id_Not_Available,discharge_disposition_id_Still_patient_referred_to_this_institution,discharge_disposition_id_Transferred_to_another_medical_facility,admission_source_id_Not_Available,admission_source_id_Referral,admission_source_id_Transferred_from_another_health_care_facility,diag_1_Circulatory,diag_1_Congenital_Anomalies,diag_1_Diabetes,diag_1_Digestive,diag_1_Endocrine__Nutritional__Metabolic__Immunity,diag_1_External_causes_of_injury,diag_1_Genitourinary,diag_1_Infectious_and_Parasitic,diag_1_Injury_and_Poisoning,diag_1_Mental_Disorders,diag_1_Musculoskeletal_System_and_Connective_Tissue,diag_1_Neoplasms,diag_1_Nervous,diag_1_Other_Symptoms,diag_1_Pregnancy__Childbirth,diag_1_Respiratory,diag_1_Sense_Organs,diag_1_Skin_and_Subcutaneous_Tissue,diag_2_Circulatory,diag_2_Congenital_Anomalies,diag_2_Diabetes,diag_2_Digestive,diag_2_Endocrine__Nutritional__Metabolic__Immunity,diag_2_External_causes_of_injury,diag_2_Genitourinary,diag_2_Infectious_and_Parasitic,diag_2_Injury_and_Poisoning,diag_2_Mental_Disorders,diag_2_Musculoskeletal_System_and_Connective_Tissue,diag_2_Neoplasms,diag_2_Nervous,diag_2_Not_Required,diag_2_Other_Symptoms,diag_2_Pregnancy__Childbirth,diag_2_Respiratory,diag_2_Sense_Organs,diag_2_Skin_and_Subcutaneous_Tissue,diag_3_Circulatory,diag_3_Congenital_Anomalies,diag_3_Diabetes,diag_3_Digestive,diag_3_Endocrine__Nutritional__Metabolic__Immunity,diag_3_External_causes_of_injury,diag_3_Genitourinary,diag_3_Infectious_and_Parasitic,diag_3_Injury_and_Poisoning,diag_3_Mental_Disorders,diag_3_Musculoskeletal_System_and_Connective_Tissue,diag_3_Neoplasms,diag_3_Nervous,diag_3_Not_Required,diag_3_Other_Symptoms,diag_3_Pregnancy__Childbirth,diag_3_Respiratory,diag_3_Sense_Organs,diag_3_Skin_and_Subcutaneous_Tissue,max_glu_serum__300,max_glu_serum_None,max_glu_serum_Norm,A1Cresult__8,A1Cresult_None,A1Cresult_Norm,insulin_treatment_insulin_only,insulin_treatment_no_med,insulin_treatment_other_meds
0,1,85,4,56,0,4,9,-2,-2,-2,0,-2,-2,-2,-2,-2,-2,1,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1
1,1,75,1,46,3,13,8,-2,-2,-2,-2,-2,-2,-2,-2,-2,-2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
2,1,55,5,52,1,23,5,-2,-2,-2,-2,-2,-2,-2,-2,0,-2,1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0
3,1,65,5,27,2,28,8,-2,-2,-2,-2,-2,1,-2,-2,0,-2,1,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0
4,0,85,11,73,0,23,9,-2,-2,-2,-2,-2,-2,-2,-2,1,-2,1,3,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0


In [ ]:
X_dum.shape, y.shape

((50000, 97), (50000,))

In [ ]:
model = LGBMClassifier(random_state = 0, n_jobs = -1)

# Create an SFS object
sbs = SFS(estimator = model,
          k_features = (1, 97),
          forward = False,       # Set forward to True when we want to perform SFS
          scoring = 'recall',
          cv = 3)               # The number of cross-validations to perform is 5

# Train SFS with our dataset
sbs = sbs.fit(X_dum, y)

TypeError: ignored

In [ ]:
# Show the performance of each subset of features considered by SFS
sbs_results = pd.DataFrame.from_dict(sbs.subsets_).T.sort_values(by = 'avg_score', ascending = False) 
sbs_results

In [ ]:
sbs_results.to_csv('/content/drive/My Drive/PGPDSE/Capstone/CSV Files/50K_LGBM_Backward_Results_Saurabh.csv', index = False)